Po volbách 2025 zde bude nutné zvýšit rozsah období na range (1,11)

In [33]:
import os
import polars as pl
import requests

In [34]:
df = pl.read_parquet(os.path.join("data","projevy.parquet")).sort(by='datum')

In [35]:
df.filter(pl.col('mluvci').str.contains('Babiš')).select(pl.col(['mluvci','mluvci_id'])).unique()

mluvci,mluvci_id
str,str
"""Poslanec Miloš Babiš""","""/sqw/detail.sqw?id=6151"""
"""Předseda vlády ČR Andrej Babiš""","""https://www.vlada.cz/cz/clenov…"
"""Poslanec Andrej Babiš""","""/sqw/detail.sqw?id=6150"""
"""Poslankyně Andrea Babišová""","""/sqw/detail.sqw?id=6428"""
"""Místopředseda vlády ČR a minis…","""http://www.vlada.cz/cz/clenove…"


In [36]:
len(df)

801985

In [37]:
len(df.filter(pl.col('mluvci_id').str.contains('/sqw/detail')))

477038

In [38]:
df.filter(pl.col('mluvci').str.contains('Marek Benda')).select(pl.col(['mluvci','mluvci_id'])).unique()

mluvci,mluvci_id
str,str
"""Poslanec Marek Benda""","""/sqw/detail.sqw?o=3&id=4"""
"""Poslanec Marek Benda""","""/cgi-bin/win/sqw/hlasy.sqw?G=5…"
"""Poslanec Marek Benda""","""/sqw/hlasy.sqw?G=10693"""
"""Poslanec Marek Benda""","""/sqw/hlasy.sqw?G=5531"""
"""Poslanec Marek Benda""","""/sqw/hlasy.sqw?G=2757"""
…,…
"""Poslanec Marek Benda""","""/sqw/historie.sqw?T=1&O=3"""
"""Poslanec Marek Benda""","""/sqw/hlasy.sqw?G=4554"""
"""Poslanec Marek Benda""","""/sqw/hlasy.sqw?G=2138"""


In [39]:
mluvci_raw = df.select(pl.col("mluvci_id")).unique().to_series().to_list()

In [40]:
len(mluvci_raw)

2174

In [41]:
import random

In [42]:
random.sample(mluvci_raw, 20)

['/sqw/detail.sqw?id=6535',
 '/sqw/detail.sqw?id=437',
 '/sqw/detail.sqw?id=5717&sfo=6',
 '/sqw/hlasy.sqw?G=6807',
 '/ff/c5/33/01.htm',
 '/sqw/detail.sqw?id=5701&sfo=6',
 '/sqw/hlasy.sqw?G=4302',
 '/sqw/hlasy.sqw?G=5296',
 '/sqw/hlasy.sqw?G=5559',
 '/sqw/detail.sqw?id=6751',
 '/sqw/p.sqw?P=529',
 'http://wtd.vlada.cz/scripts/detail.php?id=9214',
 '/sqw/hlasy.sqw?G=5168',
 '/sqw/detail.sqw?id=6467',
 '/sqw/detail.sqw?id=6052',
 'http://vlada.cz/cz/clenove-vlady/judr--pavel-blazek--ph-d---97064/',
 '/sqw/p.sqw?P=474',
 '/sqw/detail.sqw?id=5707&sfo=6',
 '/sqw/detail.sqw?id=6765',
 '/sqw/hlasy.sqw?G=4331']

In [43]:
kam = "downloads/poslanectvo"

In [44]:
os.makedirs(kam, exist_ok=True)

In [45]:
def stahni_poslance(url):
    if url:
        if not '/sqw/detail.sqw?' in url:
            return None
        else:
            id = url.split('id=')[-1].split('&')[0]
            for dodatek in range(0,10):
                soubor = f'{id}_{dodatek}.html'
                if dodatek == 0:
                    print(f'Stahuji id {id}')
                if soubor not in stazene:
                    try:
                        if 'psp.cz' not in url:
                            url = 'https://www.psp.cz' + url
                        if 'https://' not in url:
                            url = 'https://' + url
                        response = requests.get(url)
                        if response.status_code == 200:
                            with open(os.path.join(kam, soubor), 'w+', encoding='utf-8') as poslancododatek:
                                poslancododatek.write(response.text)
                    except:
                        print(f"chyba: {url}")

In [46]:
def stahni_poslance(url):
    if url:
        if not '/sqw/detail.sqw?' in url:
            return None
        else:
            id = url.split('id=')[-1].split('&')[0]
            
            for obdobi in range(1,10):
                soubor = f'{id}_{obdobi}.html'
                if obdobi == 1:
                    print(f'Stahuji id {id}')
                if soubor not in stazene:
                    try:
                        if 'psp.cz' not in url:
                            url = 'https://www.psp.cz' + url
                        if 'https://' not in url:
                            url = 'https://' + url
                        response = requests.get(f'{url}&o={obdobi}')
                        if response.status_code == 200:
                            with open(os.path.join(kam, soubor), 'w+', encoding='utf-8') as poslancoobdobi:
                                poslancoobdobi.write(response.text)
                    except Exception as e:
                        print(f"{e}: {url}")

In [47]:
stazene = set(os.listdir(kam))

print(f"{len(stazene)} souborů staženo")

for m in mluvci_raw:
    stahni_poslance(m)

print("hotovo dvacet")

9387 souborů staženo
Stahuji id 5798
Stahuji id 369
Stahuji id 5868
Stahuji id 346
Stahuji id tep
Stahuji id 6113
Stahuji id 6474
Stahuji id 405
Stahuji id 6449
Stahuji id 309
Stahuji id 6593
Stahuji id 392
Stahuji id 5908
Stahuji id 312
Stahuji id 6211
Stahuji id 5445
Stahuji id 6485
Stahuji id 6754
Stahuji id 6182
Stahuji id 6167
Stahuji id 6428
Stahuji id 6500
Stahuji id 4737
Stahuji id 5505
Stahuji id 5476
Stahuji id 307
Stahuji id 269
Stahuji id 5909
Stahuji id 6337
Stahuji id 5465
Stahuji id 5379
Stahuji id 5273
Stahuji id 250
Stahuji id 5314
Stahuji id 6170
Stahuji id 345
Stahuji id 5938
Stahuji id 63
Stahuji id 6488
Stahuji id 247
Stahuji id 5906
Stahuji id 6453
Stahuji id 6402
Stahuji id 5728
Stahuji id 357
Stahuji id 5978
Stahuji id 420
Stahuji id 5461
Stahuji id 67
Stahuji id 349
Stahuji id 5740
Stahuji id 6146
Stahuji id 6180
Stahuji id 5937
Stahuji id 6044
Stahuji id 6758
Stahuji id 1016
Stahuji id 16
Stahuji id 6870
Stahuji id 6796
Stahuji id 455
Stahuji id 253
Stahuji id